In [1]:
import uproot
import numpy as np
import awkward as ak
import math as m
import matplotlib.pyplot as plt
import matplotlib as mpl
import mplhep as hep
import os
import numba as nb
import boost_histogram as bh
import yaml
import json
from coffea import hist as chist
from coffea.nanoevents.methods import vector

import sys
import time

from coffea.util import save
from coffea.util import load

In [2]:
def mkdir(path):
    '''
    创建指定的文件夹
    :param path: 文件夹路径，字符串格式
    :return: True(新建成功) or False(文件夹已存在，新建失败)
    '''
    # 引入模块
    import os

    # 去除首位空格
    path = path.strip()
    # 去除尾部 \ 符号
    path = path.rstrip("\\")

    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists = os.path.exists(path)

    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
         # 创建目录操作函数
        os.makedirs(path)
        print(path + ' 创建成功')
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print(path + ' 目录已存在')
        return False

In [4]:
isdata = 'data'#'data' or 'mc'
output_dir = f'newid_chargeflip_hist'
mkdir(output_dir)

array_dir = f'newid_{isdata}_chargeflip'
pwd = os.popen('pwd').readlines()[0].replace('\n','')

ntotal_file = len(os.listdir(array_dir))

nevents_after_cr_filter = 0
nevents_pass = 0

nevents_real_after_cr_filter = 0
nevents_real_pass = 0

electron_ptbins=np.array([20,40,60,80,100])
electron_etabins=np.array([0,1.0,1.5,2.0,2.5])
electron_highest_pt_bin_center=(electron_ptbins[len(electron_ptbins)-1]+electron_ptbins[len(electron_ptbins)-2])/2
cf_electrons_hist = chist.Hist("All",
           chist.Bin("x", "abs_eta", electron_etabins),
           chist.Bin("y", "$\mathrm{p_T[GeV]}$", electron_ptbins),
          )
cf_electrons_hist_pass = chist.Hist("Pass",
           chist.Bin("x", "abs_eta", electron_etabins),
           chist.Bin("y", "$\mathrm{p_T[GeV]}$", electron_ptbins),
          )


for i,rootfile in enumerate(os.listdir(array_dir)):
    print("\r", end="")
    if not os.path.exists(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'x_array'):
        print('%d/%d wrong'%(i+1, ntotal_file))
    else:
        if isdata=='data':
            try:
                x_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'x_array')
                y_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'y_array')
                cf_electrons_hist.fill(x=x_array,y=y_array)
                nevents_after_cr_filter += len(x_array)
            except:
                pass
            try:
                x_pass_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'x_pass_array')
                y_pass_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'y_pass_array')
                cf_electrons_hist_pass.fill(x=x_pass_array,y=y_pass_array)
                nevents_pass += len(x_pass_array)
            except:
                pass
        else:
            try:
                x_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'x_array')
                y_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'y_array')
                gen_weight_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'gen_weight_array')
                if lepton=='electron':
                    weight_array = gen_weight_array*1*(y_array<25)*weight_lowpt + gen_weight_array*1*(y_array>=25)*weight_highpt
                else:
                    weight_array = gen_weight_array*1*(y_array<20)*weight_lowpt + gen_weight_array*1*(y_array>=20)*weight_highpt

                cf_electrons_hist.fill(x=x_array,y=y_array,weight=weight_array)
                nevents_after_cr_filter += len(x_array)
                nevents_real_after_cr_filter += np.sum(weight_array)
            except:
                pass
            try:
                x_pass_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'x_pass_array')
                y_pass_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'y_pass_array')
                gen_weight_pass_array = ak.from_parquet(pwd+'/'+array_dir+'/'+str(i+1)+'/'+'gen_weight_pass_array')

                if lepton=='electron':
                    weight_pass_array = gen_weight_pass_array*1*(y_pass_array<25)*weight_lowpt + gen_weight_pass_array*1*(y_pass_array>=25)*weight_highpt
                else:
                    weight_pass_array = gen_weight_pass_array*1*(y_pass_array<20)*weight_lowpt + gen_weight_pass_array*1*(y_pass_array>=20)*weight_highpt

                cf_electrons_hist_pass.fill(x=x_pass_array,y=y_pass_array,weight=weight_pass_array)
                nevents_pass += len(x_pass_array)
                nevents_real_pass += np.sum(weight_pass_array)
            except:
                pass
        
    print("Progress: %d/%d"%(i+1, ntotal_file),end="")
    print("\t(%d,%d)"%(nevents_after_cr_filter,nevents_pass),end="")
    if isdata=='mc':
        print("\t(%.2f,%.2f)"%(nevents_real_after_cr_filter,nevents_real_pass),end="")
    sys.stdout.flush()

newid_chargeflip_hist 目录已存在
1/946 wrong
2/946 wrong/946	(0,0)
3/946 wrong/946	(0,0)
4/946 wrong/946	(0,0)
5/946 wrong/946	(0,0)
6/946 wrong/946	(0,0)
7/946 wrong/946	(0,0)
8/946 wrong/946	(0,0)
9/946 wrong/946	(0,0)
10/946 wrong946	(0,0)
11/946 wrong/946	(0,0)
12/946 wrong/946	(0,0)
13/946 wrong/946	(0,0)
14/946 wrong/946	(0,0)
15/946 wrong/946	(0,0)
16/946 wrong/946	(0,0)
17/946 wrong/946	(0,0)
18/946 wrong/946	(0,0)
19/946 wrong/946	(0,0)
20/946 wrong/946	(0,0)
21/946 wrong/946	(0,0)
22/946 wrong/946	(0,0)
23/946 wrong/946	(0,0)
24/946 wrong/946	(0,0)
25/946 wrong/946	(0,0)
26/946 wrong/946	(0,0)
27/946 wrong/946	(0,0)
28/946 wrong/946	(0,0)
29/946 wrong/946	(0,0)
30/946 wrong/946	(0,0)
31/946 wrong/946	(0,0)
32/946 wrong/946	(0,0)
33/946 wrong/946	(0,0)
34/946 wrong/946	(0,0)
35/946 wrong/946	(0,0)
36/946 wrong/946	(0,0)
37/946 wrong/946	(0,0)
38/946 wrong/946	(0,0)
39/946 wrong/946	(0,0)
40/946 wrong/946	(0,0)
41/946 wrong/946	(0,0)
42/946 wrong/946	(0,0)
43/946 wrong/946	(0,0)
44/

346/946 wrong/946	(0,0)
347/946 wrong/946	(0,0)
348/946 wrong/946	(0,0)
349/946 wrong/946	(0,0)
350/946 wrong/946	(0,0)
351/946 wrong/946	(0,0)
352/946 wrong/946	(0,0)
353/946 wrong/946	(0,0)
354/946 wrong/946	(0,0)
355/946 wrong/946	(0,0)
356/946 wrong/946	(0,0)
357/946 wrong/946	(0,0)
358/946 wrong/946	(0,0)
359/946 wrong/946	(0,0)
360/946 wrong/946	(0,0)
361/946 wrong/946	(0,0)
362/946 wrong/946	(0,0)
363/946 wrong/946	(0,0)
364/946 wrong/946	(0,0)
365/946 wrong/946	(0,0)
366/946 wrong/946	(0,0)
367/946 wrong/946	(0,0)
368/946 wrong/946	(0,0)
369/946 wrong/946	(0,0)
370/946 wrong/946	(0,0)
371/946 wrong/946	(0,0)
372/946 wrong/946	(0,0)
373/946 wrong/946	(0,0)
374/946 wrong/946	(0,0)
375/946 wrong/946	(0,0)
376/946 wrong/946	(0,0)
377/946 wrong/946	(0,0)
378/946 wrong/946	(0,0)
379/946 wrong/946	(0,0)
380/946 wrong/946	(0,0)
381/946 wrong/946	(0,0)
382/946 wrong/946	(0,0)
383/946 wrong/946	(0,0)
384/946 wrong/946	(0,0)
385/946 wrong/946	(0,0)
386/946 wrong/946	(0,0)
387/946 wrong/94

687/946 wrong/946	(0,0)
688/946 wrong/946	(0,0)
689/946 wrong/946	(0,0)
690/946 wrong/946	(0,0)
691/946 wrong/946	(0,0)
692/946 wrong/946	(0,0)
693/946 wrong/946	(0,0)
694/946 wrong/946	(0,0)
695/946 wrong/946	(0,0)
696/946 wrong/946	(0,0)
697/946 wrong/946	(0,0)
698/946 wrong/946	(0,0)
699/946 wrong/946	(0,0)
700/946 wrong/946	(0,0)
701/946 wrong/946	(0,0)
702/946 wrong/946	(0,0)
703/946 wrong/946	(0,0)
704/946 wrong/946	(0,0)
705/946 wrong/946	(0,0)
706/946 wrong/946	(0,0)
707/946 wrong/946	(0,0)
708/946 wrong/946	(0,0)
709/946 wrong/946	(0,0)
710/946 wrong/946	(0,0)
711/946 wrong/946	(0,0)
712/946 wrong/946	(0,0)
713/946 wrong/946	(0,0)
714/946 wrong/946	(0,0)
715/946 wrong/946	(0,0)
716/946 wrong/946	(0,0)
717/946 wrong/946	(0,0)
718/946 wrong/946	(0,0)
719/946 wrong/946	(0,0)
720/946 wrong/946	(0,0)
721/946 wrong/946	(0,0)
722/946 wrong/946	(0,0)
723/946 wrong/946	(0,0)
724/946 wrong/946	(0,0)
725/946 wrong/946	(0,0)
726/946 wrong/946	(0,0)
727/946 wrong/946	(0,0)
728/946 wrong/94